In [36]:
import librosa
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn import preprocessing
import wav_to_spec
import os

In [49]:
PATHSAMPLES = '../data/training_list.txt'
NSAMPLES = 3000
SOUNDS =np.array(['yes','no','right','five','nine'])
path_out = '/home/josearangos/Documentos/Projects/Voice_Recognition/data/spectrograms'

In [50]:
def get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS):
    ROOTPATH= 'data/'
    data = pd.read_csv(PATHSAMPLES, sep=" ", header=None)
    data = data.values #Convertir en numpy array
    labels = []
    speakers  = []
    paths = []
    
    
    for path in data:
        pathSplit =path[0].split('/')
        speakerName = pathSplit[1].split('_')[0]
        word = pathSplit[0]
        isWord = np.where(SOUNDS == word)
        if len(isWord[0]) != 0:
            label = SOUNDS[isWord[0][0]]
            labels.append(label)
            speakers.append(speakerName)
            paths.append(ROOTPATH+path[0])
    NSAMPLES = NSAMPLES
    first = labels.index(SOUNDS[0])
    last = first +NSAMPLES
    matrixMajor = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
    for i in range(1,np.size(SOUNDS,0)):
        first = labels.index(SOUNDS[i])
        last = first +NSAMPLES
        clasei = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
        matrixMajor = np.append(matrixMajor,clasei,axis=0)
    return matrixMajor

In [51]:
path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)

In [52]:
path_word_speakerId.shape

(15000, 3)

In [53]:
path_word_speakerId

array([['data/yes/004ae714_nohash_0.wav', 'yes', '004ae714'],
       ['data/yes/004ae714_nohash_1.wav', 'yes', '004ae714'],
       ['data/yes/00970ce1_nohash_0.wav', 'yes', '00970ce1'],
       ...,
       ['data/nine/f2b8fc18_nohash_0.wav', 'nine', 'f2b8fc18'],
       ['data/nine/f2e9b610_nohash_0.wav', 'nine', 'f2e9b610'],
       ['data/nine/f2e9b610_nohash_1.wav', 'nine', 'f2e9b610']],
      dtype='<U33')

In [54]:
testP = path_word_speakerId[1,0]
testP

'data/yes/004ae714_nohash_1.wav'

In [55]:
spect = wav_to_spec.wav2spect('../'+testP)

In [56]:
spect.shape

(99, 161)

In [ ]:
speakers =[]
words = []
spectrograms = []

 #Encode etiqueta del path audio,speaker y la palabra
leSpeaker = preprocessing.LabelEncoder()
leWord =  preprocessing.LabelEncoder()

#Encode
leSpeaker.fit(path_word_speakerId[:,2])
leWord.fit(path_word_speakerId[:,1])


for p in path_word_speakerId:
    name = p[0].split('/')[-1].split('.')[0]+'.jpg'
    word = p[1]
    speaker_id = p[-1]
    
 #Transformación del encoder
    labelSpeaker = leSpeaker.transform([speaker_id])
    labelWord= leWord.transform([word])     
    
    spect = wav_to_spec.wav2spect('../'+p[0]) 
    spectrograms.append(spect)
    speakers.append(labelSpeaker[0])
    words.append(labelWord[0])

In [ ]:
df = pd.DataFrame([spectrograms,speakers,words],index=['Spectrogram','speaker','word']).T

In [ ]:
df

In [48]:
df.to_hdf('/home/josearangos/Documentos/Projects/Voice_Recognition/data/spectrograms/spectrograms_15000.hd5', key = 'description')

/home/josearangos/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['Spectrogram', 'speaker', 'word']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)


Rellenar de cero 

https://www.kaggle.com/alphasis/light-weight-cnn-lb-0-74